# International Faculty Development Program on _“Quantum Artificial Intelligence”_

### Quantum $K$ Nearest Neighbour (QKNN)

We have datapoints with defined labels. A new datapoin encountered, for which label is unknown. What label should be assigned to this datapoints?

![KNN](https://miro.medium.com/v2/resize:fit:720/format:webp/0*ItVKiyx2F3ZU8zV5)

In [ ]:
from sklearn.datasets import make_blobs
from matplotlib.pyplot import subplots, scatter, show

# Create data points
train_size = 15
test_size = 50
n_features = 2
centers = [[0, 0], [3, 4]]
data, labels = make_blobs(n_samples = train_size+test_size, n_features = n_features, centers = centers, cluster_std = 0.5)

# Visualize the data
_, ax = subplots(1, 1)
scatter(data[:,0], data[:,1], c = labels, cmap = 'viridis')
show()

In [ ]:
# Normalize the dataset
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler(feature_range=(0, 1))
min_max_scaler.fit(data)
data = min_max_scaler.transform(data)

# Visualize the data
_, ax = subplots(1, 1)
scatter(data[:,0], data[:,1], c = labels, cmap = 'viridis')
show()

In [ ]:
# Split train and test data
from sklearn.model_selection import train_test_split

data_train, data_test, label_train, label_test = train_test_split(data, labels, train_size = train_size, test_size = test_size)

In [ ]:
from numpy import zeros, ceil, log2, sqrt, kron

# Construct quantum state for training data
N = int(ceil(log2(train_size)))
psi = zeros(2**(n_features+N))

for i in range(train_size):
    # Encode index
    i_vec = zeros(2**N)
    i_vec[i] = 1

    # Encode train data
    x = data_train[i, :]
    for j in range(n_features):
        dummy = [sqrt(x[j]), sqrt(1- x[j])]
        if j == 0:
            x_vec = dummy
        else:
            x_vec = kron(dummy, x_vec)
    psi_i = kron(x_vec, i_vec)
    psi += psi_i
psi /= sqrt(train_size)

In [ ]:
# Select a sample test sample for classification
phi_test = data_test[0]
print(f'Choosen testing data: {phi_test}')

# Create the encoded feature vector
for i in range(n_features):
    phi_i = [sqrt(phi_test[i]), sqrt(1- phi_test[i])]
    if i == 0:
        phi = phi_i
    else:
        phi = kron(phi_i, phi)  

In [ ]:
from qiskit import QuantumRegister, QuantumCircuit

# Create circuit
index = QuantumRegister(N, 'index')
train = QuantumRegister(n_features, 'train')
test = QuantumRegister(n_features, 'test')
ancilla = QuantumRegister(1, 'ancilla')

qknn = QuantumCircuit(index, train, test, ancilla)

# Initialize the training register
qknn.initialize(psi, index[0:N] + train[0:n_features])

# Initialize the test register
qknn.initialize(phi, test)
qknn.barrier()

# Draw qknn
qknn.draw('mpl')

In [ ]:
def swap_test(n):
    '''
    `N`: Number of qubits of the quantum registers.
    '''
    a = QuantumRegister(n, 'a')
    b = QuantumRegister(n, 'b')
    c = QuantumRegister(1, 'd')
    
    # Quantum Circuit
    qc_swap = QuantumCircuit(name = ' SWAP \nTest')
    qc_swap.add_register(a)
    qc_swap.add_register(b)
    qc_swap.add_register(c)
    
    qc_swap.h(c)
    for i in range(n):
        qc_swap.cswap(c, a[i], b[i])
            
    qc_swap.h(c)    
    
    return qc_swap

In [ ]:
from qiskit import ClassicalRegister

# Apply SWAP Test
qknn.append(swap_test(n_features), train[0:n_features] + test[0:n_features] + [ancilla[0]])
qknn.barrier()

# Add classical register and measure
meas = ClassicalRegister(N+1, 'meas')
qknn.add_register(meas)

# Measure the qubits
qknn.measure(index[0::] + ancilla[0::], meas)

# Draw qknn
qknn.decompose().draw('mpl')

In [ ]:
from qiskit import transpile
from qiskit_aer import AerSimulator

# Run the circuit and get counts
backend = AerSimulator()
qknn = transpile(qknn, backend)
counts = backend.run(qknn, shots = 10000).result().get_counts()
counts

In [ ]:
# Seperate counts for each training data
result = zeros((train_size, 3))
for count in counts:
    i_dec = int(count[1:], 2)
    phase = int(count[0], 2)
    if phase == 0:
        result[i_dec][0] += counts[count]
    else:
        result[i_dec][1] += counts[count]
result

In [ ]:
# Calculate distance
for i in range(train_size):        
    prob = result[i][0]/(result[i][0] + result[i][1])
    result[i][2] = sqrt(2*prob - 1)
result

In [ ]:
from statistics import mode

# Find the indexes of minimum distance
k = 10
k_min_dist = result[:, 2].argsort()[::-1][:k]

# Determine the class of the test sample
label_pred = mode(label_train[k_min_dist])
label_exp = label_test[0]
print('Predicted class of the test sample is {}.'.format(label_pred))
print('Expected class of the test sample is {}.'.format(label_exp))

In [ ]:
def q_knn(test_index, psi, k = 10):
    '''Performs Quantum KNN'''
    from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
    from qiskit_aer import AerSimulator
    from statistics import mode
    
    # Select a sample test sample for classification
    phi_test = data_test[0]

    # Create the encoded feature vector
    for i in range(n_features):
        phi_i = [sqrt(phi_test[i]), sqrt(1- phi_test[i])]
        if i == 0:
            phi = phi_i
        else:
            phi = kron(phi_i, phi)

    # Create circuit
    index = QuantumRegister(N, 'index')
    train = QuantumRegister(n_features, 'train')
    test = QuantumRegister(n_features, 'test')
    ancilla = QuantumRegister(1, 'ancilla')

    qknn = QuantumCircuit(index, train, test, ancilla)

    # Initialize the training register
    qknn.initialize(psi, index[0:N] + train[0:n_features])

    # Initialize the test register
    qknn.initialize(phi, test)
    qknn.barrier()
    
    # Apply SWAP Test
    qknn.append(swap_test(n_features), train[0:n_features] + test[0:n_features] + [ancilla[0]])
    qknn.barrier()

    # Add classical register and measure
    meas = ClassicalRegister(N+1, 'meas')
    qknn.add_register(meas)

    # Measure the qubits
    qknn.measure(index[0::] + ancilla[0::], meas)

    # Run the circuit and get counts
    backend = AerSimulator()
    qknn = transpile(qknn, backend)
    counts = backend.run(qknn, shots = 10000).result().get_counts()
    
    # Seperate counts for each training data
    result = zeros((train_size, 3))
    for count in counts:
        i_dec = int(count[1:], 2)
        phase = int(count[0], 2)
        if phase == 0:
            result[i_dec][0] += counts[count]
        else:
            result[i_dec][1] += counts[count]
            
    # Calculate distance
    for i in range(train_size):        
        prob = result[i][0]/(result[i][0] + result[i][1])
        result[i][2] = sqrt(2*prob - 1)

    # Find the indexes of minimum distance
    k_min_dist = result[:, 2].argsort()[::-1][:k]

    # Determine the class of the test sample
    label_pred = mode(label_train[k_min_dist])
    label_exp = label_test[0]

    return label_pred, label_exp
 
label_pred = []
label_exp = [] 

p = 0
f = 0
for test_index in range(len(data_test)):
    label_pred, label_exp = q_knn(test_index, psi)
    if label_pred == label_exp:
        p += 1
    else:
        f += 1
    
print('Model accuracy is {}%.'.format(p/(p+f) *100))

#### Reference

1. https://www.geeksforgeeks.org/k-nearest-neighbours/
2. Li, J., Lin, S., Yu, K. et al. Quantum K-nearest neighbor classification algorithm based on Hamming distance. Quantum Inf Process **21**, 18 (2022).
3. https://qiskit-quantum-knn.readthedocs.io/en/latest/